In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [2]:
#downloading the training data
training_data= datasets.FashionMNIST(root="data", train=True, download =True, transform=ToTensor())

In [3]:
test_data=datasets.FashionMNIST(root="data",train=False, download= True, transform =ToTensor())

In [4]:
batch_size=64
train_loader=DataLoader(training_data, batch_size=batch_size)
test_loader=DataLoader(test_data, batch_size=batch_size)
for X,y in test_loader:
    print(f"shape of X [N,C,H,W]:{X.shape}")
    print(f"shape of y: {y.shape} {y.dtype}")
    break

shape of X [N,C,H,W]:torch.Size([64, 1, 28, 28])
shape of y: torch.Size([64]) torch.int64


In [28]:
device=("cuda" if torch.cuda.is_available() else "cpu")
print("using {device} device")
class NNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten=nn.Flatten()
        self.lrstack=nn.Sequential(nn.Linear(28*28,512),nn.ReLU(),nn.Linear(512,512), nn.ReLU(),nn.Linear(512,512), nn.ReLU(),nn.Linear(512,10))
    def forward(self,x):
        x=self.flatten(x)
        logits=self.lrstack(x)
        return logits
model=NNetwork().to(device)
print(model)

using {device} device
NNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (lrstack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=512, bias=True)
    (5): ReLU()
    (6): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [29]:
lossfn=nn.CrossEntropyLoss()

optimizer= torch.optim.Adam(model.parameters(), lr=1e-3)

In [30]:
def train(dataloader,model,  lossfn, optimizer):
    size=len(dataloader.dataset)
    model.train()
    for batch, (X,y) in enumerate(dataloader):
        X,y=X.to(device),y.to(device)
        pred=model(X)
        loss=lossfn(pred,y)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        if batch %100 ==0:
            loss,current=loss.item(),(batch + 1)*len(X)
            print(f"loss: {loss:>7f} [{current:>5d}/{size: >5d}]" )
            

In [31]:
def test(dataloader, model, lossfn):
    size=len(dataloader.dataset)
    num_batches=len(dataloader)
    model.eval()
    test_loss,correct=0,0
    with torch.no_grad():
        for X,y in dataloader:
            X,y=X.to(device),y.to(device)
            pred=model(X)
            testloss=lossfn(pred,y).item()
            correct+=(pred.argmax(1)==y).type(torch.float).sum().item()
        testloss/=num_batches
        correct/=size
        print(f"test accuracy \n accuracy {(100*correct):>0.1f}%, avg_loss: {testloss:>8f}\n")

In [ ]:
epochs=8
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_loader, model, lossfn, optimizer)
    test(test_loader, model, lossfn)
print("done")

Epoch 1
-------------------------------
loss: 2.302819 [   64/60000]
loss: 0.570744 [ 6464/60000]
loss: 0.439343 [12864/60000]
loss: 0.505735 [19264/60000]
loss: 0.593849 [25664/60000]
loss: 0.431428 [32064/60000]
loss: 0.432754 [38464/60000]
loss: 0.522378 [44864/60000]
loss: 0.507286 [51264/60000]
loss: 0.501687 [57664/60000]
test accuracy 
 accuracy 84.6%, avg_loss: 0.001525

Epoch 2
-------------------------------
loss: 0.278987 [   64/60000]
loss: 0.370292 [ 6464/60000]
loss: 0.283166 [12864/60000]
loss: 0.364155 [19264/60000]
loss: 0.521654 [25664/60000]
loss: 0.369076 [32064/60000]
loss: 0.294671 [38464/60000]
loss: 0.507705 [44864/60000]
loss: 0.427833 [51264/60000]
loss: 0.466593 [57664/60000]
test accuracy 
 accuracy 84.4%, avg_loss: 0.001258

Epoch 3
-------------------------------
loss: 0.267472 [   64/60000]
loss: 0.315883 [ 6464/60000]
loss: 0.239612 [12864/60000]
loss: 0.315798 [19264/60000]
loss: 0.401068 [25664/60000]
loss: 0.334614 [32064/60000]
loss: 0.278379 [38464/

In [10]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


In [11]:
model = NNetwork().to(device)
model.load_state_dict(torch.load("model.pth"))

<All keys matched successfully>

In [16]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[10][0], test_data[10][1]
with torch.no_grad():
    x = x.to(device)
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Pullover", Actual: "Coat"
